In [5]:
import pandas as pd
import re
from sklearn.model_selection import train_test_split

#load the dataset
data = pd.read_csv("./recipedataset.csv")
data.head()


,Prompt,Generated Recipe
0,Generate a dairy-free recipe for lunch with yo...,"Dish: Ingredients: yogurt, chickpeas, spinach,..."
1,Generate a dairy-free recipe for dinner with g...,"Dish: Ingredients: ginger, olive oil, tomato, ..."
2,Generate a vegetarian recipe for dinner with c...,"Dish: Ingredients: cucumber, potato, tofu, bre..."
3,Generate a dairy-free recipe for lunch with le...,"Dish: Ingredients: lentils, basil, spinach, on..."
4,Generate a vegetarian recipe for dinner with b...,"Dish: Ingredients: basil, lemongrass, pasta, b..."


In [6]:
def clean_text(text):
  text.lower()
  text = re.sub(r'[^a-zA-Z]'," ",text)
  return text

data["CleanedPrompt"] = data["Prompt"].apply(clean_text)
data["CleanedGenerated Recipe"] = data["Generated Recipe"].apply(clean_text)

train_data,test_data = train_test_split(data,test_size=0.2,random_state=42)

In [7]:
!pip install datasets

In [21]:
from transformers import T5Tokenizer, T5ForConditionalGeneration,Trainer,TrainingArguments
from datasets import Dataset

# Initialize the tokenizer and model
tokenizer = T5Tokenizer.from_pretrained("t5-base")
train_encodings = tokenizer(list(train_data["CleanedPrompt"]), return_tensors="pt", padding=True, truncation=True,max_length=123)
train_labels = tokenizer(list(train_data["CleanedGenerated Recipe"]), return_tensors="pt", padding=True, truncation=True,max_length=123)

test_encoding = tokenizer(list(train_data["CleanedPrompt"]), return_tensors="pt", padding=True, truncation=True,max_length=123)
test_labels = tokenizer(list(train_data["CleanedGenerated Recipe"]), return_tensors="pt", padding=True, truncation=True,max_length=123)

train_dataset = Dataset.from_dict({
    'input_ids': train_encodings['input_ids'],
    'attention_mask': train_encodings['attention_mask'],
    'labels': train_labels['input_ids']
})

test_dataset = Dataset.from_dict({
    'input_ids': test_encoding['input_ids'],
    'attention_mask': test_encoding['attention_mask'],
    'labels': test_labels['input_ids']
})



In [24]:
model = T5ForConditionalGeneration.from_pretrained("t5-base")
training_args = TrainingArguments(
        output_dir='./results',  # Output directory
    num_train_epochs=5,  # Number of training epochs
    per_device_train_batch_size=8,  # Batch size for training
    per_device_eval_batch_size=8,  # Batch size for evaluation
    warmup_steps=500,  # Warmup steps
    weight_decay=0.01,  # Weight decay
    logging_dir='./logs',  # Logging directory
    logging_steps=10,
    evaluation_strategy="steps",  # Evaluate during training
    eval_steps=500,  # Evaluate every 500 steps
    save_steps=1000,  # Save model checkpoints every 1000 steps
    load_best_model_at_end=True,  # Load the best model when finished
)
trainer = Trainer(
    model = model,
    train_dataset=train_dataset,
    eval_dataset =test_dataset,
    args=training_args,
)
trainer.train()

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Step,Training Loss,Validation Loss
500,0.007300,0.000015
1000,0.000200,0.000003
1500,0.000200,0.000002


There were missing keys in the checkpoint model loaded: ['encoder.embed_tokens.weight', 'decoder.embed_tokens.weight', 'lm_head.weight'].


TrainOutput(global_step=1500, training_loss=0.4942823014521661, metrics={'train_runtime': 450.195, 'train_samples_per_second': 26.655, 'train_steps_per_second': 3.332, 'total_flos': 428173516800000.0, 'train_loss': 0.4942823014521661, 'epoch': 5.0})

In [ ]:
model.save_pretrained("./recipe_model")
tokenizer.save_pretrained("./recipe_model")

In [33]:
import torch
from transformers import T5ForConditionalGeneration,T5Tokenizer

model = T5ForConditionalGeneration.from_pretrained("./recipe_model")
tokenizer = T5Tokenizer.from_pretrained("./recipe_model")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

def generate_recipe(prompt,model,tokenizer,max_length):
  input_ids = clean_text(prompt)
  input_ids = tokenizer.encode("recipe: "+input_ids,return_tensors="pt",max_length=max_length,truncation=True)
  input_ids = input_ids.to(device)
  output = model.generate(input_ids=input_ids,max_length=max_length,num_beams=5,no_repeat_ngram_size=2,early_stopping=True)
  recipe = tokenizer.decode(output[0],skip_special_tokens=True)
  return recipe

In [34]:
prompt = "Generate a vegan dessert recipe with chocolate and almonds"
print(generate_recipe(prompt,model,tokenizer,512))

Dish Ingredients chocolate, almonds, and macadamia nuts Instructions Combine chocolate and almond extract add almond milk cook thoroughly and serve hot


In [35]:
import shutil
from google.colab import files

model_dir = "./recipe_model"

shutil.make_archive("recipe_model", 'zip', model_dir)
files.download('recipe_model.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>